<h1> Goals </h1>
 <ul>
  <li>Load the ZHVI data for each ZIP code, selecting only Philadelphia ZIP codes.</li>
  <li>Calculate the annual average ZHVI for each ZIP code in Philadelphia and each year.</li>
  <li>Identify the ZIP code with the maximum value for each year.</li>
</ul> 



<h1> Import packges and data</h1>

In [2]:



import pandas as pd
import numpy as np



zillow_raw = pd.read_csv("ZHVI_all.csv")
zillow_raw.sort_values(by='City')



,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31
17237,74899,17593,38601,Zip,MS,MS,Abbeville,Oxford,Lafayette County,NaN,...,137021.0,136729.0,136477.0,136733.0,136870.0,136823.0,137248.0,137553.0,138337.0,138917.0
11156,73820,11277,36310,Zip,AL,AL,Abbeville,Dothan,Henry County,NaN,...,97587.0,97637.0,97730.0,97925.0,98125.0,98167.0,98095.0,97921.0,97972.0,97897.0
4816,88633,4857,70510,Zip,LA,LA,Abbeville,Lafayette,Vermilion Parish,NaN,...,145346.0,145841.0,146858.0,147573.0,148514.0,149236.0,150090.0,150294.0,150482.0,150284.0
8095,70603,8175,29620,Zip,SC,SC,Abbeville,Greenwood,Abbeville County,49657.0,...,139015.0,140089.0,141131.0,142505.0,143671.0,145004.0,145901.0,146953.0,147823.0,148726.0
17317,71354,17679,31001,Zip,GA,GA,Abbeville,NaN,Wilcox County,41194.0,...,77205.0,77861.0,78652.0,79194.0,79559.0,79849.0,80102.0,80532.0,80992.0,81568.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20762,82545,21540,55991,Zip,MN,MN,Zumbro Falls,Rochester,Wabasha County,116013.0,...,315743.0,318834.0,321700.0,324670.0,326877.0,329492.0,332203.0,334740.0,336974.0,338989.0
13227,82546,13390,55992,Zip,MN,MN,Zumbrota,Red Wing,Goodhue County,NaN,...,256618.0,259171.0,262831.0,266471.0,270833.0,273835.0,276352.0,279098.0,283775.0,289773.0
18202,67864,18672,23898,Zip,VA,VA,Zuni,Virginia Beach-Norfolk-Newport News,Isle of Wight County,88648.0,...,241485.0,243112.0,245609.0,248189.0,250208.0,251815.0,253423.0,255282.0,257041.0,259088.0
24224,80836,25500,52079,Zip,IA,IA,Zwingle,NaN,Jackson County,NaN,...,217380.0,218533.0,219333.0,220350.0,221549.0,222742.0,223903.0,225449.0,227362.0,230149.0


In [3]:
#Getting just Philly, PA  values had to use & becuase there were non-PA cities with the name 'Philadelphia'

#I did it this way first, not sure if this is correct 
##zillow = pd.read_csv("ZHVI_all.csv", index_col ="City")
##zillow_philly = zillow.loc["Philadelphia"]
##zillow_philly.head()

philly = ['Philadelphia']
state = ['PA']
philly_selection = zillow_raw['City'].isin(philly) & zillow_raw['State'].isin(state)
zillow_philly = zillow_raw.loc[philly_selection]
zillow_philly.tail()


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31
8517,65820,8603,19153,Zip,PA,PA,Philadelphia,Philadelphia-Camden-Wilmington,Philadelphia County,60211.0,...,179337.0,182999.0,186055.0,189217.0,192263.0,195940.0,199309.0,204155.0,209825.0,216462.0
8864,65785,8954,19118,Zip,PA,PA,Philadelphia,Philadelphia-Camden-Wilmington,Philadelphia County,210937.0,...,626674.0,632711.0,637923.0,642873.0,648301.0,655267.0,663281.0,671902.0,680375.0,685303.0
9124,65770,9217,19102,Zip,PA,PA,Philadelphia,Philadelphia-Camden-Wilmington,Philadelphia County,104342.0,...,355872.0,355508.0,355443.0,355482.0,355189.0,355229.0,354801.0,354897.0,355176.0,355060.0
10580,65794,10696,19127,Zip,PA,PA,Philadelphia,Philadelphia-Camden-Wilmington,Philadelphia County,74618.0,...,279592.0,282981.0,286069.0,288718.0,291665.0,294161.0,297464.0,301175.0,305426.0,308595.0
10693,65804,10809,19137,Zip,PA,PA,Philadelphia,Philadelphia-Camden-Wilmington,Philadelphia County,51664.0,...,194206.0,197041.0,199194.0,200296.0,201505.0,203441.0,205520.0,208074.0,210043.0,211348.0


In [4]:
#wanted to confirm what i needed to get rid of
len(zillow_philly)
print(zillow_philly.columns)

Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       'State', 'City', 'Metro', 'CountyName', '1996-01-31',
       ...
       '2020-10-31', '2020-11-30', '2020-12-31', '2021-01-31', '2021-02-28',
       '2021-03-31', '2021-04-30', '2021-05-31', '2021-06-30', '2021-07-31'],
      dtype='object', length=316)


In [5]:
#removing unwanted data, everything but region name 



removed_col = ['RegionID', 'SizeRank', 'RegionType', 'StateName',
       'State', 'City', 'Metro', 'CountyName']
zphilly_trim = zillow_philly.drop(removed_col, axis=1)
print(zphilly_trim.columns)



Index(['RegionName', '1996-01-31', '1996-02-29', '1996-03-31', '1996-04-30',
       '1996-05-31', '1996-06-30', '1996-07-31', '1996-08-31', '1996-09-30',
       ...
       '2020-10-31', '2020-11-30', '2020-12-31', '2021-01-31', '2021-02-28',
       '2021-03-31', '2021-04-30', '2021-05-31', '2021-06-30', '2021-07-31'],
      dtype='object', length=308)


In [6]:
#had an issue with zipcodes from outside of PA so i wanted to make sure that wasn't happening again
zphilly_trim.tail()

,RegionName,1996-01-31,1996-02-29,1996-03-31,1996-04-30,1996-05-31,1996-06-30,1996-07-31,1996-08-31,1996-09-30,...,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31
8517,19153,60211.0,60389.0,60603.0,60986.0,61283.0,61411.0,61498.0,61637.0,61882.0,...,179337.0,182999.0,186055.0,189217.0,192263.0,195940.0,199309.0,204155.0,209825.0,216462.0
8864,19118,210937.0,210457.0,209849.0,208570.0,207757.0,207275.0,207760.0,207852.0,208165.0,...,626674.0,632711.0,637923.0,642873.0,648301.0,655267.0,663281.0,671902.0,680375.0,685303.0
9124,19102,104342.0,103401.0,102811.0,101745.0,101410.0,101365.0,101284.0,101250.0,101212.0,...,355872.0,355508.0,355443.0,355482.0,355189.0,355229.0,354801.0,354897.0,355176.0,355060.0
10580,19127,74618.0,75181.0,75619.0,76459.0,77517.0,78527.0,79394.0,79742.0,80135.0,...,279592.0,282981.0,286069.0,288718.0,291665.0,294161.0,297464.0,301175.0,305426.0,308595.0
10693,19137,51664.0,51748.0,51910.0,52003.0,51971.0,51690.0,51538.0,51436.0,51496.0,...,194206.0,197041.0,199194.0,200296.0,201505.0,203441.0,205520.0,208074.0,210043.0,211348.0



<h1>Question</h1>

The first time around I kept a lot of the columns and so I did the melt as the code below. 
But when I only listed "RegionName" as the id_vars, RegionID showed up in the date column after the melt. Why?

`zphilly_clean = pd.melt(
  #zphilly_trim, id_vars=["RegionID", "SizeRank", "RegionName", "State", "City"], value_name="ZHVI", var_name="Date")`



In [7]:
#melting the data and renaming column



zphilly_clean = pd.melt(
    zphilly_trim, id_vars=["RegionName"], value_name="ZHVI", var_name="Date"
).rename(columns={'RegionName':'ZipCode'})

zphilly_clean.tail()


,ZipCode,Date,ZHVI
14117,19153,2021-07-31,216462.0
14118,19118,2021-07-31,685303.0
14119,19102,2021-07-31,355060.0
14120,19127,2021-07-31,308595.0
14121,19137,2021-07-31,211348.0


In [8]:

len(zphilly_clean)


14122

In [9]:
len(zphilly_trim.columns)

308

<h1> Another Question</h1>

Before melting, when each observation was a column, there were 308 columns. After melting when each observation became a row, there are 15043 rows. Shouldn't these numbers be more or less the same? **Why does my new df post-melting have so many more observations?**

In [10]:
#Convert to date
zphilly_clean['Date'] = pd.to_datetime(zphilly_clean['Date'])
zphilly_clean['Date'].head()

0   1996-01-31
1   1996-01-31
2   1996-01-31
3   1996-01-31
4   1996-01-31
Name: Date, dtype: datetime64[ns]

In [11]:
#separate date out to month and year, add new columns

zphilly_clean['Month'] = zphilly_clean['Date'].dt.month
zphilly_clean['Year'] = zphilly_clean['Date'].dt.year 

In [12]:
zphilly_clean.head()

,ZipCode,Date,ZHVI,Month,Year
0,19143,1996-01-31,53524.0,1,1996
1,19111,1996-01-31,84371.0,1,1996
2,19124,1996-01-31,44683.0,1,1996
3,19120,1996-01-31,56842.0,1,1996
4,19104,1996-01-31,83638.0,1,1996


In [13]:
#grouping by year and taking the average in fell swoop. i was about to cry at this point trying to wrap my head around this

annual_ZHVI = zphilly_clean.groupby(['ZipCode', 'Year'])['ZHVI'].mean()
print(type(annual_ZHVI))


<class 'pandas.core.series.Series'>


In [14]:
annual_ZHVI.head()

ZipCode  Year
19102    1996    102083.666667
         1997    105159.583333
         1998    109694.166667
         1999    130578.083333
         2000    155851.916667
Name: ZHVI, dtype: float64

In [16]:
annual_ZHVI = annual_ZHVI.reset_index()

##if there are no parameters for reset_index, does it just go back to what it was before? how does it know what it was before?

In [17]:
annual_ZHVI.tail(n=20)

,ZipCode,Year,ZHVI
1176,19154,2002,113527.083333
1177,19154,2003,130633.000000
1178,19154,2004,150713.416667
1179,19154,2005,179095.833333
1180,19154,2006,201504.333333
1181,19154,2007,204215.500000
1182,19154,2008,202383.000000
1183,19154,2009,193933.833333
1184,19154,2010,193575.083333
1185,19154,2011,188770.416667


In [18]:
#in trying to figure out how to isolate all the max values I found a fn that would tell me where max vals were in ea. col
#in order to make that work for me, I needed to make my year values into columns and my zipcodes the index,
#with ZHVI as my values of each
ZHVI_year_col = annual_ZHVI.pivot(index='ZipCode', columns='Year', values='ZHVI')
ZHVI_year_col.head()

Year,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
ZipCode,,,,,,,,,,,,,,,,,,,,,
19102,102083.666667,105159.583333,109694.166667,130578.083333,155851.916667,186316.750000,221209.500000,251282.833333,291917.750000,337231.166667,...,320891.333333,329256.500000,342656.000000,349828.166667,351868.333333,364323.583333,362492.750000,354211.666667,353973.833333,355119.142857
19103,135875.166667,139942.750000,146878.416667,166813.166667,199830.500000,241850.666667,281760.666667,324316.500000,369505.416667,418260.083333,...,394008.583333,404174.500000,423867.583333,441557.250000,443572.833333,463998.583333,481253.000000,484753.833333,487778.333333,501827.571429
19104,82832.000000,77473.166667,75920.250000,75536.333333,70613.916667,64964.083333,56112.000000,67023.166667,88020.666667,109810.416667,...,90462.250000,102556.333333,104861.000000,124551.083333,125702.666667,134900.333333,159991.916667,172998.083333,190154.666667,216556.571429
19106,132362.666667,135182.333333,140895.583333,156755.416667,184584.750000,220336.333333,250752.750000,283342.916667,317207.500000,364139.833333,...,327170.333333,334803.833333,346828.500000,355016.583333,362062.416667,378798.916667,392337.583333,393448.833333,397594.583333,411933.000000
19107,99782.583333,101476.166667,106144.083333,123148.666667,141244.000000,172964.833333,210014.916667,245191.083333,274325.750000,317467.916667,...,283247.333333,292033.583333,299863.000000,295667.166667,296365.000000,314488.250000,326885.083333,323932.000000,328040.500000,339629.571429


In [19]:
#code to find the max val in each column
#sad face all my zipcodes are gone but I have a solution.
#I just need to merge this table with my annual_ZHVI that does have both the max vals (hidden somewhere in there)
#and the desperately needed zipcodes
Annual_maxValues = ZHVI_year_col.max()
print(Annual_maxValues)

Year
1996    208438.000000
1997    215315.750000
1998    212398.250000
1999    223437.000000
2000    253985.750000
2001    283068.916667
2002    314700.000000
2003    361547.000000
2004    412069.666667
2005    473240.416667
2006    514634.750000
2007    531150.666667
2008    515431.916667
2009    501020.000000
2010    490081.250000
2011    462616.333333
2012    450877.750000
2013    460501.916667
2014    475473.166667
2015    499032.666667
2016    510706.500000
2017    533362.583333
2018    562148.916667
2019    590992.500000
2020    609685.166667
2021    663900.285714
dtype: float64


In [20]:
#BUT sad face I cannot merge because I'm having trouble renaming the column in the above format.
#so if i can get the data back to how it was before I should be good to go
Annual_maxValues = Annual_maxValues.reset_index()
Annual_maxValues.head()



,Year,0
0,1996,208438.00
1,1997,215315.75
2,1998,212398.25
3,1999,223437.00
4,2000,253985.75


In [21]:
#Succes and now i can rename the col so I will be able to merge with my other series 
maxvals= Annual_maxValues.rename(columns={Annual_maxValues.columns[1]:'ZHVI'})
print(maxvals)

    Year           ZHVI
0   1996  208438.000000
1   1997  215315.750000
2   1998  212398.250000
3   1999  223437.000000
4   2000  253985.750000
5   2001  283068.916667
6   2002  314700.000000
7   2003  361547.000000
8   2004  412069.666667
9   2005  473240.416667
10  2006  514634.750000
11  2007  531150.666667
12  2008  515431.916667
13  2009  501020.000000
14  2010  490081.250000
15  2011  462616.333333
16  2012  450877.750000
17  2013  460501.916667
18  2014  475473.166667
19  2015  499032.666667
20  2016  510706.500000
21  2017  533362.583333
22  2018  562148.916667
23  2019  590992.500000
24  2020  609685.166667
25  2021  663900.285714



    With my column with ZHVI now renamed to match  then took my maxvals data (which unforunately lost it's zipcode data) and merged it with a table that had all the ZVHI values and zipcodes. I merged by ZHVI and got rid of all values that did not match the maxmimums per year.

In [39]:
#merging the two tables 
zipmax = pd.merge(maxvals, annual_ZHVI, on=['ZHVI'], how='inner')
zipmax.head(n=26)

,Year_x,ZHVI,ZipCode,Year_y
0,1996,208438.000000,19118,1996
1,1997,215315.750000,19118,1997
2,1998,212398.250000,19118,1998
3,1999,223437.000000,19118,1999
4,2000,253985.750000,19118,2000
5,2001,283068.916667,19118,2001
6,2002,314700.000000,19118,2002
7,2003,361547.000000,19118,2003
8,2004,412069.666667,19118,2004
9,2005,473240.416667,19118,2005


<h1> Checking my work.</h1>

I was concerned when all the max ZHVI values for every year came from the same zipcode. I'm not entirely sure that this rules out error, but I think it verifies my answers.

In [37]:
#to seelct a value in a column based on it's numerical value to verify what I did worked? 
#df.loc[df['column_name'] == some_value]
ZHVI_year_col[2021] >  663899

ZipCode
19102    False
19103    False
19104    False
19106    False
19107    False
19111    False
19114    False
19115    False
19116    False
19118     True
19119    False
19120    False
19121    False
19122    False
19123    False
19124    False
19125    False
19126    False
19127    False
19128    False
19129    False
19130    False
19131    False
19132    False
19133    False
19134    False
19135    False
19136    False
19137    False
19138    False
19139    False
19140    False
19141    False
19142    False
19143    False
19144    False
19145    False
19146    False
19147    False
19148    False
19149    False
19150    False
19151    False
19152    False
19153    False
19154    False
Name: 2021, dtype: bool